Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28, 1) (20000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [17]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    print('conv1:', conv.shape)
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    print('conv2:', conv.shape)
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print('reshape', reshape.shape)
    print('layer3', image_size // 4 * image_size // 4 * depth, ", ", num_hidden)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print('hidden', hidden.shape)
    print('layer4', num_hidden, ", ", num_labels)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

conv1: (16, 14, 14, 16)
conv2: (16, 7, 7, 16)
reshape (16, 784)
layer3 784 ,  64
hidden (16, 64)
layer4 64 ,  10
conv1: (10000, 14, 14, 16)
conv2: (10000, 7, 7, 16)
reshape (10000, 784)
layer3 784 ,  64
hidden (10000, 64)
layer4 64 ,  10
conv1: (10000, 14, 14, 16)
conv2: (10000, 7, 7, 16)
reshape (10000, 784)
layer3 784 ,  64
hidden (10000, 64)
layer4 64 ,  10


In [18]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.156060
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.887339
Minibatch accuracy: 37.5%
Validation accuracy: 46.8%
Minibatch loss at step 100: 1.574034
Minibatch accuracy: 62.5%
Validation accuracy: 73.0%
Minibatch loss at step 150: 0.571855
Minibatch accuracy: 87.5%
Validation accuracy: 77.6%
Minibatch loss at step 200: 1.211986
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%
Minibatch loss at step 250: 0.820270
Minibatch accuracy: 75.0%
Validation accuracy: 76.4%
Minibatch loss at step 300: 0.717101
Minibatch accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 350: 0.972488
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.391628
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.482101
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.508608
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [21]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [22]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.383420
Minibatch accuracy: 0.0%
Validation accuracy: 15.0%
Minibatch loss at step 50: 1.842992
Minibatch accuracy: 31.2%
Validation accuracy: 30.3%
Minibatch loss at step 100: 1.594509
Minibatch accuracy: 62.5%
Validation accuracy: 69.4%
Minibatch loss at step 150: 0.737580
Minibatch accuracy: 75.0%
Validation accuracy: 75.8%
Minibatch loss at step 200: 1.180061
Minibatch accuracy: 68.8%
Validation accuracy: 77.8%
Minibatch loss at step 250: 0.733332
Minibatch accuracy: 81.2%
Validation accuracy: 79.8%
Minibatch loss at step 300: 0.810301
Minibatch accuracy: 75.0%
Validation accuracy: 80.8%
Minibatch loss at step 350: 0.917763
Minibatch accuracy: 68.8%
Validation accuracy: 81.1%
Minibatch loss at step 400: 0.384814
Minibatch accuracy: 87.5%
Validation accuracy: 82.3%
Minibatch loss at step 450: 0.442888
Minibatch accuracy: 87.5%
Validation accuracy: 81.3%
Minibatch loss at step 500: 0.342677
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [38]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
#   layer3_weights = tf.Variable(tf.truncated_normal(
#       [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [256, num_hidden], stddev=0.1))

  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    #hidden = tf.nn.dropout(hidden, 0.5)
    pool = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    conv = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    
    #hidden = tf.nn.dropout(hidden, 0.5)
    pool = tf.nn.avg_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    pool = tf.nn.dropout(pool, 0.5)
    shape = pool.get_shape().as_list()
    reshape = tf.reshape(pool, [shape[0], shape[1] * shape[2] * shape[3]])
    
    # assign layer3_weights
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [39]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.073675
Minibatch accuracy: 6.2%
Validation accuracy: 9.8%
Minibatch loss at step 50: 1.859795
Minibatch accuracy: 31.2%
Validation accuracy: 28.7%
Minibatch loss at step 100: 2.037742
Minibatch accuracy: 37.5%
Validation accuracy: 48.4%
Minibatch loss at step 150: 1.087922
Minibatch accuracy: 62.5%
Validation accuracy: 55.2%
Minibatch loss at step 200: 1.350351
Minibatch accuracy: 56.2%
Validation accuracy: 61.6%
Minibatch loss at step 250: 1.147450
Minibatch accuracy: 56.2%
Validation accuracy: 61.6%
Minibatch loss at step 300: 1.141829
Minibatch accuracy: 50.0%
Validation accuracy: 67.5%
Minibatch loss at step 350: 0.841565
Minibatch accuracy: 87.5%
Validation accuracy: 70.8%
Minibatch loss at step 400: 0.767838
Minibatch accuracy: 87.5%
Validation accuracy: 70.5%
Minibatch loss at step 450: 1.057801
Minibatch accuracy: 68.8%
Validation accuracy: 69.0%
Minibatch loss at step 500: 0.848038
Minibatch accuracy: 75.0%
Validation accuracy: 70.7%
Min

In [9]:
batch_size = 128 #16
patch_size = 5
depth = 20 #16
num_hidden = 500 #64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  stddev1 = np.sqrt(2.0 / (patch_size * patch_size * num_channels))
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=stddev1))
  #layer1_biases = tf.Variable(tf.zeros([depth]))
  layer1_biases = tf.Variable(tf.truncated_normal([depth], stddev=np.sqrt(2.0 / depth)))

  stddev2 = np.sqrt(2.0 / (patch_size * patch_size * depth))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=stddev2))
  #layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer2_biases = tf.Variable(tf.truncated_normal([depth], stddev=np.sqrt(2.0 / depth)))

  stddev3 = np.sqrt(2.0 / (image_size // 4 * image_size // 4 * depth))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=stddev3))
  #layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer3_biases = tf.Variable(tf.truncated_normal([num_hidden], stddev=np.sqrt(2.0 / num_hidden)))
    
  stddev4 = np.sqrt(2.0 / num_hidden)
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=stddev4))
  #layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  layer4_biases = tf.Variable(tf.truncated_normal([num_labels], stddev=np.sqrt(2.0 / num_labels)))
  
  # Model.
  def model(data):
    layer1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    print('layer1', layer1)
    relu1 = tf.nn.relu(layer1 + layer1_biases)
    pool1 = tf.nn.max_pool(relu1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
    
    layer2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    print('layer2', layer2)
    relu2 = tf.nn.relu(layer2 + layer2_biases)
    pool2 = tf.nn.max_pool(relu2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
#     if dropout:
#         pool2 = tf.nn.dropout(pool2, 0.5)
    
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    layer3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
#     if dropout:
#         layer3 = tf.nn.dropout(layer3, 0.5)
    
    return tf.matmul(layer3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))
  print('done')

layer1 Tensor("Conv2D:0", shape=(128, 28, 28, 20), dtype=float32)
layer2 Tensor("Conv2D_1:0", shape=(128, 14, 14, 20), dtype=float32)
layer1 Tensor("Conv2D_2:0", shape=(10000, 28, 28, 20), dtype=float32)
layer2 Tensor("Conv2D_3:0", shape=(10000, 14, 14, 20), dtype=float32)
layer1 Tensor("Conv2D_4:0", shape=(10000, 28, 28, 20), dtype=float32)
layer2 Tensor("Conv2D_5:0", shape=(10000, 14, 14, 20), dtype=float32)
done


In [11]:
num_steps = 3001 #5001

print('labels', train_labels.shape[0])

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    #TODO: randomize this
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

labels 20000
Initialized
Minibatch loss at step 0: 2.687935
Minibatch accuracy: 6.2%
Validation accuracy: 7.7%
Minibatch loss at step 50: 1.399287
Minibatch accuracy: 68.8%
Validation accuracy: 67.0%
Minibatch loss at step 100: 1.019126
Minibatch accuracy: 73.4%
Validation accuracy: 77.1%
Minibatch loss at step 150: 0.730290
Minibatch accuracy: 80.5%
Validation accuracy: 79.2%
Minibatch loss at step 200: 0.817790
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Minibatch loss at step 250: 0.556743
Minibatch accuracy: 85.2%
Validation accuracy: 80.8%
Minibatch loss at step 300: 0.765972
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 350: 0.696263
Minibatch accuracy: 83.6%
Validation accuracy: 80.8%
Minibatch loss at step 400: 0.686123
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Minibatch loss at step 450: 0.913282
Minibatch accuracy: 71.9%
Validation accuracy: 81.4%
Minibatch loss at step 500: 0.596469
Minibatch accuracy: 82.0%
Validation accura